# Utilisation de l'API HUBEAU

A priori, il n'y a plus de documentation sur l'extraction des données XML depuis Vigicrues (mais ça marche toujours le 01/11)
Les infos sur Hubeau sont disponibles:
* http://hubeau.eaufrance.fr/page/api-hydrometrie
* https://api.gouv.fr/api/api_hubeau_hydrometrie.html
* https://www.data.gouv.fr/fr/datasets/hauteurs-deau-et-debits-des-cours-deau-observes-en-temps-reel-aux-stations-du-reseau-vigicrues/

In [ ]:
import requests
import json
import pandas as pd
import geopandas as gpd
import io
import bokeh

## Récupération des stations

Utilisation de GET stations

In [ ]:
# url = "http://hubeau.eaufrance.fr/api/v1/hydrometrie/referentiel/stations?en_service=true&format=json&size=20"
fields = "code_site,type_station"
# url = f"http://hubeau.eaufrance.fr/api/v1/hydrometrie/referentiel/stations?en_service=true&format=json&size=10000&fields={fields}"
url = f"http://hubeau.eaufrance.fr/api/v1/hydrometrie/referentiel/stations?en_service=true&format=geojson&size=10000"
r = requests.get(url)

In [ ]:
gdf = gpd.read_file(io.BytesIO(r.content), driver="GeoJSON")

In [ ]:
gdf_deb = gdf.query("type_station == 'DEB'").reset_index()

In [ ]:
def requete_temps_reel_station(code_entite, size, grandeur_hydro="Q"):
    obs_tr_csv_url = "http://hubeau.eaufrance.fr/api/v1/hydrometrie/observations_tr.csv"
    obs_tr_csv_params = {
        "code_entite": code_entite,
        "grandeur_hydro": grandeur_hydro,
        "size": size,
        "fields": ["date_obs", "resultat_obs"],
    }
    try:
        obs_tr_response = requests.get(
            obs_tr_csv_url,
            obs_tr_csv_params,
            timeout=10,
        )
    except requests.exceptions.Timeout:
        return
    
    if not obs_tr_response.ok:
        return

    response_str = obs_tr_response.text

    if not response_str:
        return

    return (code_entite, response_str)

In [ ]:
def recuperer_data_temps_reel_station(code_entite, size=10_000, grandeur_hydro="Q"):

    res_requete = requete_temps_reel_station(code_entite, size, grandeur_hydro)
    if res_requete is None:
        print("Pas de données à transformer en DataFrame.")
        return

    _, response_str = res_requete
    
    df_tr = pd.read_csv(io.StringIO(response_str), sep=";")

    # Les débits sont certainement stockés sous la forme d'integers dans
    # la base de données pour réduire la taille. Il faut diviser par 1000
    # pour obtenir un débit en m3/s.
    df_tr["resultat_obs"] = df_tr["resultat_obs"] / 1000
    df_tr["date_obs"] = pd.to_datetime(df_tr["date_obs"], utc=True)
    df_tr = df_tr.set_index("date_obs", drop=True)
    # La première valeur retournée par la requête est la dernière observation ;)
    df_tr = df_tr.sort_index()
    # Conversion pour la métropole avec la région Europe/Paris (attention aux stations dans les DomTom!)
    df_tr.index = df_tr.index.tz_convert("Europe/Paris")
    df_tr.index = df_tr.index.rename("Temps")
    
    column_name = "Débit [m³/s]" if grandeur_hydro == "Q" else "Hauteur [m]"
    df_tr = df_tr.rename(columns={"resultat_obs": column_name})
              
    return df_tr

## Threads 

In [ ]:
import concurrent.futures

In [ ]:
codes = list(gdf_deb["code_station"])

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=None) as executor:
    station_responses = executor.map(requete_temps_reel_station, codes, [1]*len(codes))
station_responses = list(station_responses)

In [ ]:
stations_with_data = [
    info[0] for info in station_responses if info is not None 
]

## Que les stations avec des données

In [ ]:
gdf_deb_red = gdf_deb.query("code_station in @stations_with_data").reset_index()

## Dashboard 

In [ ]:
import panel as pn
import holoviews as hv
import datashader.geo
import hvplot.pandas

In [ ]:
osm = hv.element.tiles.OSM()

In [ ]:
x, y = datashader.geo.lnglat_to_meters(gdf_deb_red.longitude_station, gdf_deb_red.latitude_station)
gdf_deb_projected = pd.DataFrame(gdf_deb_red.join([pd.DataFrame({'easting': x}), pd.DataFrame({'northing': y})]))

In [ ]:
key_dimensions   = ["easting", "northing"]
value_dimensions = ["libelle_station", "libelle_cours_eau", "libelle_commune"]
gdf_deb_projected_table = hv.Table(gdf_deb_projected, key_dimensions, value_dimensions)

In [ ]:
hover_map = bokeh.models.HoverTool(
    tooltips=[
#         ( 'date',   '@date{%F}'            ),
#         ( 'close',  '$@{adj close}{%0.2f}' ), # use @{ } for field names with spaces
#         ( 'volume', '@volume{0.00 a}'      ),
        ("Station", "@libelle_station"),
        ("Cours d'eau", "@libelle_cours_eau"),
    ],

#     formatters={
#         'date'      : 'datetime', # use 'datetime' formatter for 'date' field
#     },

    # display a tooltip whenever the cursor is vertically in line with a glyph
#     mode='vline'
)

gdf_deb_projected_points = (
    gdf_deb_projected_table.to.points(
        ["easting", "northing"],
        ["libelle_station", "libelle_cours_eau"]
)
#     .options(tools=["hover"], width=800)
    .options(
        tools=[hover_map],
        marker="diamond",
        size=10,
    )
)
stations_q = gdf_deb_projected_points

In [ ]:
selection_stream = hv.streams.Selection1D(source=stations_q)

In [ ]:
def labelled_callback(index):
    if len(index) == 0:
        return hv.Text(x=0,y=0, text="")
    first_index = index[0] # Pick only the first one if multiple are selected
    row = gdf_deb_projected.iloc[first_index]
    return hv.Text(x=row["easting"] ,y=row["northing"] ,text=row["code_station"]).opts(color='white')

labeller = hv.DynamicMap(labelled_callback, streams=[selection_stream])

In [ ]:
# station_map = (esri * stations_q.options(tools=['tap']) * labeller).options(hv.opts.Scatter(tools=['hover']))
station_map = (
    osm
    * stations_q.options(
        tools=["tap", hover_map],
        nonselection_alpha=0.4,
        selection_color="red",
        frame_height=600,
        aspect=1
    )
)

In [ ]:
empty_plot = hv.Curve(kdims="Temps", vdims="Débit [m³/s]", data=[])
error_plot = hv.Curve(kdims="Temps", vdims="Débit [m³/s]", data=[]).options(title="Erreur")

hover_line = bokeh.models.HoverTool(
    tooltips=[
        ("Temps", "@Temps{%F %T}"),
        ("Débit [m³/s]", "@{Débit [m³/s]}{0.00}"),
    ],

    formatters={
        "Temps": 'datetime', # use 'datetime' formatter for 'date' field
    },

#     display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)

def station_tempplot_callback(index):
    if len(index) == 0:
        return empty_plot.options(title="Pas de données à afficher")
    first_index = index[0] # Pick only the first one if multiple are selected
    row = gdf_deb_projected.iloc[first_index]
    code_station = row["code_station"]
    libelle_site = row["libelle_site"]
    
    df_q = recuperer_data_temps_reel_station(code_station, grandeur_hydro="Q")
    if df_q is None:
        # TODO: Remplacer les return None dans recup_temps_reel_station par
        # des erreurs qui seraient attrapées ici, pour que ça soit plus clair.
        return error_plot
    
    plot =  df_q.hvplot.line(
    ).options(tools=[hover_line], title=f"{libelle_site} ({code_station})", padding=0.1, alpha=0.7)
    return plot

station_tempplot = hv.DynamicMap(station_tempplot_callback, streams=[selection_stream]).options(
    framewise=True,
    frame_height=300,
    frame_width=600
)

In [ ]:
def station_table_callback(index):
    if len(index) == 0:
        return hv.Table(data=[])
    first_index = index[0] # Pick only the first one if multiple are selected
    row = gdf_deb_projected.iloc[first_index]
    code_station = row["code_station"]
    
    df_q = recuperer_data_temps_reel_station(code_station, grandeur_hydro="Q")

    return df_q.hvplot.table().options(height=300)

station_table = hv.DynamicMap(station_table_callback, streams=[selection_stream]).options(
)

In [ ]:
# Dashboard
app = pn.Column(
    "# My vigicrues",
    "Sélectionner une station débitmétrique pour voir le débit mesuré et disponible à partir de l'API Hubeau.",
    "Les données sont téléchargées à chaque clic, il faut un peu patienter avant de les voir s'affichier.",
    pn.Row(station_map, pn.Tabs(("Graphique", station_tempplot), ("Données", station_table), closable=True)),
    width_policy="max"
)

In [ ]:
app.servable()